In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
184,AFG,Asia,Afghanistan,2020-07-12,34451.0,85.0,1010.0,16.0,884.985,2.183,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
310,ALB,Europe,Albania,2020-07-12,3371.0,93.0,89.0,4.0,1171.381,32.316,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
500,DZA,Africa,Algeria,2020-07-12,18712.0,904.0,1004.0,16.0,426.717,20.615,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-11,France,171454.0,0.0,30022.0,0.0,0.0,0.0
0,2020-07-11,China,85104.0,0.0,4650.0,0.0,0.0,0.0
0,2020-07-11,Italy,242978.0,0.0,34974.0,0.0,0.0,0.0
0,2020-07-11,Spain,254478.0,0.0,28420.0,0.0,0.0,0.0
0,2020-07-11,United States,3237005.0,0.0,135154.0,0.0,0.0,0.0
0,2020-07-11,World,12689392.0,0.0,566095.0,0.0,0.0,0.0
0,2020-07-11,United Kingdom,289118.0,0.0,44787.0,0.0,0.0,0.0
0,2020-07-11,Germany,199022.0,0.0,9075.0,0.0,0.0,0.0
0,2020-07-11,Iran,256001.0,0.0,12639.0,0.0,0.0,0.0
0,2020-07-11,Turkey,211943.0,0.0,5340.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-12,France,171454.0,170752.0,30022.0,30004.0,702.0,18.0
0,2020-07-12,China,85104.0,85071.0,4650.0,4641.0,33.0,9.0
0,2020-07-12,Italy,242978.0,242827.0,34974.0,34945.0,151.0,29.0
0,2020-07-11,Spain,254478.0,253908.0,28420.0,28403.0,570.0,17.0
0,2020-07-12,United States,3237005.0,3247684.0,135154.0,134814.0,-10679.0,340.0
0,2020-07-12,World,12689392.0,12698995.0,566095.0,564924.0,-9603.0,1171.0
0,2020-07-12,United Kingdom,289118.0,288953.0,44787.0,44798.0,165.0,-11.0
0,2020-07-12,Germany,199022.0,198804.0,9075.0,9063.0,218.0,12.0
0,2020-07-12,Iran,256001.0,255117.0,12639.0,12635.0,884.0,4.0
0,2020-07-12,Turkey,211943.0,211981.0,5340.0,5344.0,-38.0,-4.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")